In [1]:
import numpy as np
import matplotlib.pyplot as plt
import time
%matplotlib inline
import skimage.io as io
import math
import cv2
import queue

In [2]:
path = 'photo\p0\\'

In [3]:
# Mirror flip back depth map
img = cv2.imread(path+'back_newPng_origin.png')
img2 = cv2.flip(img,1,dst=None)
cv2.imwrite(path+'back_newPng_origin_jingxaing.png',img2)

# img = cv2.imread(path+'tps_back_newPng_origin.png')
# img2 = cv2.flip(img,1,dst=None)
# cv2.imwrite(path+'tps_back_newPng_origin_jingxaing.png',img2)


True

#### Smooth edges

In [4]:
import queue

# Starting from the center point, bfs expands outward, smoothing the depth value of the point that changes greatly
# x, y is the center of the picture, img is the depth of the picture, gap is the maximum difference 
# in the depth of adjacent elements, and diff is the reasonable range of the difference in depth.
def bfs_smooth(x,y,img,gap,diff):
    q = queue.Queue()
    visited = np.mat(np.zeros((x,y)))

    mid_x = int(x/2)
    mid_y = int(y/2)
    q.put((mid_x,mid_y))
    visited[mid_x,mid_y]=1
    
    while not q.empty():
        cx,cy = q.get()
        before = img[cx][cy]
        for i in range(-1,2):
            for j in range(-1,2):
                new_x = cx+i
                new_y = cy+j
                if(new_x<x and new_y<y and img[new_x][new_y]!=0 and visited[new_x,new_y] == 0 ):
                    visited[new_x,new_y] = 1
                    q.put((new_x,new_y))

                    if(abs(int(img[new_x][new_y])-int(before))<diff):
                        pass
                    elif(img[new_x][new_y]>before):
                        img[new_x][new_y] = before+gap
                    else:
#                         img[new_x][new_y] = before-gap
                        pass
                    

#### Preprocessing

In [5]:
img= cv2.imread(path+'newPng_origin.png',0)
# img= cv2.imread('tps_newPng_origin.png',0)
x,y=img.shape
pointdepth=[]
dp_matrix_value = np.mat(np.zeros((x,y)))

bfs_smooth(x,y,img,2,8)

count = 1
for i in range(x):
    for ii in range(y):
        if img[i][ii]==0:
            pass
        else:
            dp_matrix_value[i,ii] = count
            count+=1
            pointdepth.append([i,ii,img[i][ii]])
            
print("-----------")

back_img= cv2.imread(path+'back_newPng_origin_jingxaing.png',0)
# back_img= cv2.imread('tps_back_newPng_origin_jingxaing.png',0)
back_pointdepth=[]
back_dp_matrix_value = np.mat(np.zeros((x,y)))
count0 = count

bfs_smooth(x,y,back_img,2,8)

for i in range(x):
    for ii in range(y):
        if back_img[i][ii]==0:
            pass
        else:
            back_dp_matrix_value[i,ii] = count
            count+=1
        
            back_pointdepth.append([i,ii,285-back_img[i][ii]])

-----------


In [6]:
print(len(back_pointdepth))
print(len(pointdepth))

7214
7214


In [9]:
def get_point(x, y, z, camera_factor, camera_cx, camera_cy, camera_fx, camera_fy):
#     camera_factor Scaling factor
#     camera_cx  Camera position
#     camera_cy  Camera position
#     camera_fx  Camera focal length
#     camera_fy  Camera focal length is generally equal to camera_fx
    
#     camera_factor = 1;
#     camera_cx = 78.4;
#     camera_cy = 125.5;
#     camera_fx = 560.0;
#     camera_fy = 560.0;
    
    newz = z / camera_factor
    newx = (x - camera_cx) * newz / camera_fx;
    newy = (y - camera_cy) * newz / camera_fy;
    
    f=3
    return round(newx,f),round(newy,f),round(newz,f)


In [10]:
# Storage of edge points, special treatment
data1 = np.load(path+'small_contours.npy')
contours=[]
for i in data1:
    contours.append((i[0],i[1]))

#### Write to obj file

In [11]:
rat = 0.7
lenth = min(len(pointdepth),len(back_pointdepth))
qurt=[]
back_qurt=[]
with open(path+r'b_rat285.obj','w+') as file:
# with open(r'tps_b_rat.obj','w+') as file:
    #Geometry vertices
    cout =0
    for i in range(lenth):
        q1 = pointdepth[i]
        q2 = back_pointdepth[i]
        if(q1[2]>q2[2]):
            
            mid = (q1[2]+q2[2])/2
            pointdepth[i][2] = mid
            back_pointdepth[i][2] = mid
            
        if (q1[1],q1[0]) in contours:
            mid = (q1[2]+q2[2])/2
            pointdepth[i][2] = mid
            back_pointdepth[i][2] = mid
            
            
    for q in pointdepth:
        file.write('v '+str(q[0])+' '+str(q[1])+' '+str(q[2]*rat)+'\n')
        
    for q in back_pointdepth:
        file.write('v '+str(q[0])+' '+str(q[1])+' '+str(q[2]*rat)+'\n')
    


        
    #surface
    for i in range(x-1):
        for ii in range(y-1):
            if(img[i][ii]!=0 and img[i+1][ii]!=0 and img[i+1][ii+1]!=0 and img[i][ii+1]!=0):
                qurt.append([dp_matrix_value[i,ii],dp_matrix_value[i+1,ii],dp_matrix_value[i+1,ii+1],dp_matrix_value[i,ii+1]])

                
#     124 234

    for qu in qurt:
        file.write('f '+str(int(qu[0]))+' '+str(int(qu[1]))+' '+str(int(qu[3]))+'\n') 
        file.write('f '+str(int(qu[1]))+' '+str(int(qu[2]))+' '+str(int(qu[3]))+'\n') 
        
    
        #面
    for i in range(x-1):
        for ii in range(y-1):
            if(back_img[i][ii]!=0 and back_img[i+1][ii]!=0 and back_img[i+1][ii+1]!=0 and back_img[i][ii+1]!=0):
                back_qurt.append([back_dp_matrix_value[i,ii],back_dp_matrix_value[i+1,ii],back_dp_matrix_value[i+1,ii+1],back_dp_matrix_value[i,ii+1]])

                
#     124 234

    for qu in back_qurt:
        file.write('f '+str(int(qu[0]))+' '+str(int(qu[1]))+' '+str(int(qu[3]))+'\n') 
        file.write('f '+str(int(qu[1]))+' '+str(int(qu[2]))+' '+str(int(qu[3]))+'\n') 